In [41]:
import pandas as pd
import numpy as np
import pickle
from scipy import stats

In [42]:
with open('distance_df_v2.pickle', 'rb') as f:
    distance_df = pickle.load(f)
distance_df.head()

,station,station_lat,station_lon,venueId,venueCategory,poi_lat,poi_lon,distance
1,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a5200fea1ee3,Salad Place,40.739980,-73.986648,0.718688
11,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a5204ceb1ee3,Sushi Restaurant,40.738116,-73.987964,0.843644
14,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a52083e31ee3,Sushi Restaurant,40.737531,-73.991127,0.898471
17,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a5208ae71ee3,Cupcake Shop,40.757140,-73.993508,0.550242
19,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a520b7e51ee3,Clothing Store,40.739744,-73.994964,0.815413


In [43]:
with open('Benson_full_analysis.pickle_v2', 'rb') as f:
    station_entries_df = pickle.load(f)
station_entries_df.head()

,STATION,ENTRIES,EXITS,mean_percentage
0,1 AV,108048.545455,121148.909091,1.383897
1,103 ST,173966.090909,108841.727273,2.228176
2,103 ST-CORONA,107435.272727,65010.454545,1.376042
3,104 ST,21531.545455,8188.272727,0.275778
4,110 ST,58447.818182,44527.000000,0.748606


In [44]:
distance_by_station = distance_df.groupby(by='station', as_index=False).count()

In [45]:
distance_df.head()

,station,station_lat,station_lon,venueId,venueCategory,poi_lat,poi_lon,distance
1,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a5200fea1ee3,Salad Place,40.739980,-73.986648,0.718688
11,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a5204ceb1ee3,Sushi Restaurant,40.738116,-73.987964,0.843644
14,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a52083e31ee3,Sushi Restaurant,40.737531,-73.991127,0.898471
17,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a5208ae71ee3,Cupcake Shop,40.757140,-73.993508,0.550242
19,34 ST-HERALD SQ,40.750326,-73.988067,3fd66200f964a520b7e51ee3,Clothing Store,40.739744,-73.994964,0.815413


In [46]:
station_priorities = station_entries_df[['STATION', 'ENTRIES']].rename(columns={'ENTRIES': 'avg_entries', 'STATION': 'station'})
station_priorities.head()

,station,avg_entries
0,1 AV,108048.545455
1,103 ST,173966.090909
2,103 ST-CORONA,107435.272727
3,104 ST,21531.545455
4,110 ST,58447.818182


In [47]:
station_priorities = station_priorities.merge(distance_by_station, on='station')

In [48]:
station_priorities.head()

,station,avg_entries,station_lat,station_lon,venueId,venueCategory,poi_lat,poi_lon,distance
0,14 ST-UNION SQ,521808.545455,673,673,673,673,673,673,673
1,23 ST,546372.636364,566,566,566,566,566,566,566
2,34 ST-HERALD SQ,594431.181818,506,506,506,506,506,506,506
3,34 ST-PENN STA,784022.727273,498,498,498,498,498,498,498
4,47-50 STS ROCK,255785.818182,484,484,484,484,484,484,484


In [49]:
station_priorities['avg_entries'] = station_priorities['avg_entries'].astype(int)

In [50]:
station_priorities.drop(['station_lat', 'station_lon', 'venueId', 'venueCategory', 'poi_lat', 'poi_lon'], axis=1, inplace = True)

In [51]:
station_priorities.rename(columns={'distance':'nearby_poi_count'},inplace=True)

In [52]:
station_priorities['entry_weight'] = station_priorities['avg_entries']/station_priorities['avg_entries'].sum()

In [53]:
station_priorities['poi_weight'] = station_priorities['nearby_poi_count']/station_priorities['nearby_poi_count'].sum()

In [54]:
#station_priorities['combined_weight_product'] = station_priorities['entry_weight'] * station_priorities['poi_weight']
station_priorities['combined_share_mean'] = station_priorities[['entry_weight','poi_weight']].mean(axis=1)
station_priorities.rename(columns={'entry_weight':'entries_share', 'poi_weight':'poi_share'}, inplace=True)
#station_priorities['combined_weight_harmonic_mean'] = stats.hmean(station_priorities[['entry_weight','poi_weight']], axis=1)

In [55]:
station_priorities.sort_values('combined_share_mean', inplace=True, ascending=False)

In [56]:
station_priorities

,station,avg_entries,nearby_poi_count,entries_share,poi_share,combined_share_mean
3,34 ST-PENN STA,784022,498,0.156138,0.114667,0.135402
0,14 ST-UNION SQ,521808,673,0.103918,0.154962,0.129440
7,GRD CNTRL-42 ST,720653,483,0.143518,0.111213,0.127366
1,23 ST,546372,566,0.108810,0.130325,0.119567
2,34 ST-HERALD SQ,594431,506,0.118381,0.116509,0.117445
9,TIMES SQ-42 ST,463714,468,0.092348,0.107760,0.100054
4,47-50 STS ROCK,255785,484,0.050939,0.111444,0.081192
5,59 ST COLUMBUS,347784,318,0.069261,0.073221,0.071241
6,FULTON ST,496950,129,0.098967,0.029703,0.064335
8,PATH NEW WTC,289834,218,0.057720,0.050196,0.053958


In [57]:
#distance_df.groupby(by=['venueCategory','station'], as_index=False).count().sort_values(['venueCategory','poi_lat'], ascending=[True,False])

In [58]:
with open('final_station_priorities_v2.pickle', 'wb') as f:
    pickle.dump(station_priorities, f)
    

,station,avg_entries,nearby_poi_count,entries_share,poi_share,combined_share_mean
6,FULTON ST,496950,129,0.098967,0.029703,0.064335
8,PATH NEW WTC,289834,218,0.057720,0.050196,0.053958
5,59 ST COLUMBUS,347784,318,0.069261,0.073221,0.071241
9,TIMES SQ-42 ST,463714,468,0.092348,0.107760,0.100054
7,GRD CNTRL-42 ST,720653,483,0.143518,0.111213,0.127366
4,47-50 STS ROCK,255785,484,0.050939,0.111444,0.081192
3,34 ST-PENN STA,784022,498,0.156138,0.114667,0.135402
2,34 ST-HERALD SQ,594431,506,0.118381,0.116509,0.117445
1,23 ST,546372,566,0.108810,0.130325,0.119567
0,14 ST-UNION SQ,521808,673,0.103918,0.154962,0.129440
